In [18]:
import json
import cv2 as cv
import pickle as pkl
from typing import Dict, List
import numpy as np
from matplotlib import pyplot as plt
import os, sys
import librosa

from scipy import stats, spatial, ndimage
sys.path.insert(0, '/Users/evanpan/Documents/GitHub/EvansToolBox/Utils')
sys.path.insert(0, '/Users/evanpan/Documents/GitHub/Gaze_project')
sys.path.insert(0, "C:/Users/evansamaa/Documents/GitHub/EvansToolBox/Utils")
from Signal_processing_utils import intensity_from_signal, pitch_from_signal, sparse_key_smoothing, laplacian_smoothing
from Speech_Data_util import Sentence_word_phone_parser
from prototypes.InputDataStructures import Dietic_Conversation_Gaze_Scene_Info
from prototypes.MVP.MVP_static_saliency_list import ObjectBasedFixSaliency
from prototypes.MVP.MVP_Aversion_saliency_list import AversionSignalDrivenSaliency, CTSAversionSignalDrivenSaliency
from prototypes.MVP.MVP_look_at_point_planner import HabituationBasedPlanner, RandomPlanner, PartnerHabituationPlanner
from prototypes.MVP.MVP_eye_head_driver import HeuristicGazeMotionGenerator
from prototypes.MVP.MVP_Aversion_saliency_list import Base_Static_Saliency_List
from prototypes.EyeCatch.Saccade_model_with_internal_model import *
from prototypes.Gaze_aversion_prior.Heuristic_model import *
from prototypes.Boccignone2020.Gaze_target_planner import Scavenger_based_planner
from prototypes.Boccignone2020.Improved_gaze_target_planner import Scavenger_planner_with_nest, Scavenger_planner_simple 
from prototypes.JaliNeck.JaliNeck import NeckCurve
from prototypes.Gaze_aversion_prior.Ribhav_model import predict_aversion
from prototypes.InputDataStructures import AgentInfo, TurnTakingData
from prototypes.MVP.MVP_gaze_path_planner import Responsive_planner_simple
import pickle
import math
from datetime import datetime, timezone
import scipy.stats as ss
from Signal_processing_utils import interpolate1D, runEuro, intensity_from_signal
import numpy as np
from scipy.interpolate import interp1d

In [19]:
%load_ext autoreload
%autoreload 1
%aimport prototypes.InputDataStructures
%aimport Signal_processing_utils
%aimport Speech_Data_util
%aimport prototypes.MVP.MVP_static_saliency_list
%aimport prototypes.EyeCatch.Saccade_model_with_internal_model
%aimport prototypes.InputDataStructures
%aimport prototypes.Jin2019.EyeHeadDecomposition
%aimport prototypes.Optimization_based_head_eye_seperator.Baseline_optimization
%aimport prototypes.Boccignone2020.Improved_gaze_target_planner
%aimport prototypes.MVP.MVP_gaze_path_planner
%aimport prototypes.JaliNeck.JaliNeck


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# inputs
input_folder = "/Volumes/EVAN_DISK/MASC/Ribhav_processed_dataset/"
# input_folder = "F:/MASC/Ribhav_processed_dataset/"
# input_file = "Stranger Things’ Dacre Montgomery’s Insane _Billy_ Audition Tape ｜ GQ"
input_file = "Madelaine Petsch audition for The Prom"

input_scene_data_path = "/Users/evanpan/Documents/GitHub/Gaze_project/data/look_at_points/simplest_scene.json"
speaker_id = 1
shot_id = 1
turn_taking_threshold = 2
fps = 24
np.random.seed(speaker_id + shot_id)


In [21]:

input_file_no_space = "_".join(input_file.split(" "))
# input_file_no_space = "'" + input_file_no_space + "'"
shot_file_name = os.path.join(os.path.join(os.path.join(input_folder, "shots"), input_file), "shot_cuts.json")
video_path = os.path.join(*[input_folder, "video", input_file+".mp4"])
audio_1_path = os.path.join(os.path.join(os.path.join(input_folder, "JaliTranscript"), input_file_no_space), "shot_{}_{}.wav".format(shot_id, 1))
audio_2_path = os.path.join(os.path.join(os.path.join(input_folder, "JaliTranscript"), input_file_no_space), "shot_{}_{}.wav".format(shot_id, 2))
script_1_path = os.path.join(os.path.join(os.path.join(input_folder, "JaliTranscript"), input_file_no_space), "shot_{}_{}.txt".format(shot_id, 1))
script_2_path = os.path.join(os.path.join(os.path.join(input_folder, "JaliTranscript"), input_file_no_space), "shot_{}_{}.txt".format(shot_id, 2))
praatoutput_1_path = os.path.join(os.path.join(os.path.join(input_folder, "JaliTranscript"), input_file_no_space), "shot_{}_{}_PraatOutput.txt".format(shot_id, 1))
praatoutput_2_path = os.path.join(os.path.join(os.path.join(input_folder, "JaliTranscript"), input_file_no_space), "shot_{}_{}_PraatOutput.txt".format(shot_id, 2))
head_direction_json_path = os.path.join(*[input_folder, "pose", input_file+".pkl"])
gaze_direction_json_path = os.path.join(*[input_folder, "L2CSNet", input_file+".json"])
landmarks_mediapipe_json_path = os.path.join(*[input_folder, "faces", input_file+" faces.pkl"])
python_out_location = os.path.join(*[input_folder, "outputs", input_file_no_space+".pkl"])
scene_file_out_location = os.path.join(*[input_folder, "annotated_scene", input_file+".json"])
animation_file_path = os.path.join(*[input_folder, "outputs", input_file+".json"])
gd_file_path = os.path.join(*[input_folder, "outputs", input_file+"_gaze_deploy.json"])
st_file_path = os.path.join(*[input_folder, "outputs", input_file+"_stare.json"])
ss_file_path = os.path.join(*[input_folder, "outputs", input_file+"_static_salience.json"])

scene_data_path = os.path.join(*[input_folder, "annotated_scene", input_file+".json"])

# out_location = "/Users/evanpan/Documents/GitHub/Gaze_project/data/look_at_points/prototype2p2.pkl"
# out_location = "C:/Users/evansamaa/Documents/GitHub/Gaze_project/data/look_at_points/prototype2p2.pkl"

sementic_script_1 = Sentence_word_phone_parser(praatoutput_1_path, script_1_path)
# sementic_script_1.get_turns(turn_taking_threshold)
sementic_script_2 = Sentence_word_phone_parser(praatoutput_2_path, script_2_path)
# sementic_script_2.get_turns(turn_taking_threshold)
audio_1, sr= librosa.load(audio_1_path, sr=44100)
audio_2, sr= librosa.load(audio_2_path, sr=44100)
agentScene1 = AgentInfo(scene_data_path)
agentScene2 = AgentInfo(scene_data_path)

# Get the beat from the audio as per Rhythmic Gesticulator

In [43]:
# get the energy and derivative
audio = audio_1
audio_energy = intensity_from_signal(audio, int(sr/50))
ts = np.arange(0, audio_energy.shape[0]) / 50
daudio_dt = dx_dt(audio_energy)
Dm = 0.2
DM = 0.6
DM_frame = math.floor(DM / (ts[1] - ts[0]))

In [45]:
# iterative find audio onset between 0.2 and 0.6 seconds to identify beats
beats = [[0, False]] # start with a pseudo beat
for i in range(0, audio_energy.shape[0]):
    if daudio_dt[i] > 5:
        current_beat_t = ts[i]
        if current_beat_t - ts[beats[-1][0]] <= Dm:
            continue
        if current_beat_t - tspbeats[-1][0]] >= DM:
            # these are stored as integer indexes
            start = beats[-1][0]
            end = current_beat_t
            counter = start + DM_frame
            while counter < end:
                beats.append([counter, False])
                counter = counter + DM_frame
        beats.append([i, True])